In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

### this text is taken from Mushaf Al-Madinah Hafs narration version 18 from King Fahad you can download data and fonts from here "https://qurancomplex.gov.sa/techquran/dev/" (Unicode Uthmanic Font (Hafs Narration))
### اخذ نص هذا المصحف من مصحف المدينة الرقمي الاصدار رقم 18 برواية حفص و يمكن تنزيله من الرابط اعلاه

In [ ]:
mushaf_madinah = pd.read_csv('data/hafsData_v18.csv')
mushaf_madinah.head()

In [ ]:
mushaf_madinah.info()

In [ ]:
mushaf_madinah.rename(columns={'id':'aya_index'}, inplace= True)
mushaf_madinah.rename(columns={'aya_text_emlaey':'spoken_aya_text'}, inplace= True)
mushaf_madinah.rename(columns={'aya_text':'written_aya_text'}, inplace= True)
mushaf_madinah.rename(columns={'sora':'sura_mushaf_order'}, inplace= True)
mushaf_madinah.rename(columns={'sora_name_ar':'sura_name'}, inplace= True)
quran = mushaf_madinah

## some Quran metadata from tanzil "https://tanzil.net/docs/"

In [ ]:
tanzil_quran_metadata_root = ET.parse('data/tanzil_quran_metadata.xml').getroot()
suras = {}
for child in tanzil_quran_metadata_root.find('suras'):
    suras[int(child.attrib['index'])] = [child.attrib['order'], child.attrib['type']]
quran['sura_revel_order'] = quran['sura_mushaf_order'].apply(lambda x: int(suras[x][0]))
quran['sura_revel_place'] = quran['sura_mushaf_order'].apply(lambda x: 'مكية' if suras[x][1] == 'Meccan' else 'مدنية')

# Calculate Jomal (حساب الجُمّل Abjad numerals )
here some useful link "http://www.islamnoon.com/new/Derasat/Erhasat/e_hesab.htm"

In [ ]:
quran_letters = {'ء','أ','إ','ا','ٱ','آ','ى','ب','ت','ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ؤ','ي','ئ'}
letter_jomal = {'ء': 1 ,'أ': 1 ,'إ': 1 ,'ا': 1 ,'ٱ': 1 ,'آ': 1 ,'ى': 1 ,'ب': 2 ,'ت': 400 ,'ة': 400 ,'ث': 500 ,'ج': 3 ,'ح': 8 ,'خ': 600 ,'د': 4 ,'ذ': 700 ,'ر': 200 ,'ز': 7 ,'س': 60 ,'ش': 300 ,'ص': 90 ,'ض': 800 ,'ط': 9 ,'ظ': 900 ,'ع': 70 ,'غ': 1000 ,'ف': 80 ,'ق': 100 ,'ك': 20 ,'ل': 30 ,'م': 40 ,'ن': 50 ,'ه': 5 ,'و': 6 ,'ؤ': 6 ,'ي': 10 ,'ئ': 10}

def get_written_aya_words(aya):
    aya = aya.split()[:-1]
    if aya[0] == '۞':
        aya = aya[1:]
    if aya[-1][-1] == '۩':
        aya[-1] = aya[-1][:-1]
    return aya

def get_letters(text):
    letters = []
    for words in text.split():
        for c in words:
            if c in quran_letters:
                letters.append(c)
    return letters

def get_invariant_letters(text):
    letters = []
    for words in text.split():
        for c in words:
            if c in quran_letters:
                if c in ['ء','آ','أ','إ','ا','ى','ٱ']:
                    letters.append('ا')
                elif c in ['ة','ت',]:
                    letters.append('ت')
                elif c in ['ؤ','و']:
                    letters.append('و')
                elif c in ['ئ','ي']:
                    letters.append('ي')
                else:
                    letters.append(c)
    return letters

def calculate_text_jomal(text):
    total = 0
    for c in get_letters(text):
        total += letter_jomal[c]
    return total
    
def calculate_letters_jomal(letters):
    total = 0
    for c in letters:
        total += letter_jomal[c]
    return total

def calculate_cumulative_sum(number):
    total = 0
    for n in range(number + 1):
        total += n
    return total

def get_aya_indexes_list(aya):
    pass


# sora name featuers

In [ ]:
quran['sura_mushaf_order_cumulative'] = quran['sura_mushaf_order'].apply(lambda order: calculate_cumulative_sum(order))
quran['sura_revel_order_cumulative'] = quran['sura_revel_order'].apply(lambda order: calculate_cumulative_sum(order))
quran['letters_of_sura_name'] = quran['sura_name'].apply(lambda name: get_letters(name))
quran['letters_count_of_sura_name'] = quran['letters_of_sura_name'].apply(lambda letters: len(letters))
quran['invariant_letters_of_sura_name'] = quran['sura_name'].apply(lambda name: get_invariant_letters(name))
quran['invariant_letters_count_of_sura_name'] = quran['invariant_letters_of_sura_name'].apply(lambda letters: len(letters))
quran['jomal_of_sura_name'] = quran['letters_of_sura_name'].apply(lambda letters: calculate_letters_jomal(letters))
quran['jomal_of_sura_name_cumulative'] = quran['jomal_of_sura_name'].cumsum()
quran['unique_letters_of_sura_name'] = quran['letters_of_sura_name'].apply(lambda letters: set(letters))
quran['unique_letters_count_of_sura_name'] = quran['unique_letters_of_sura_name'].apply(lambda letters: len(letters))
quran['unique_letters_invariant_of_sura_name'] = quran['invariant_letters_of_sura_name'].apply(lambda letters: set(letters))
quran['unique_letters_invariant_count_of_sura_name'] = quran['unique_letters_invariant_of_sura_name'].apply(lambda letters: len(letters))
quran['unique_letters_jomal_of_sura_name'] = quran['unique_letters_of_sura_name'].apply(lambda letters: calculate_letters_jomal(letters))
quran['unique_letters_jomal_of_sura_name_cumulative'] = quran['unique_letters_jomal_of_sura_name'].cumsum()
quran.head(10)[['sura_mushaf_order_cumulative','sura_revel_order_cumulative','letters_of_sura_name',\
    'letters_count_of_sura_name','invariant_letters_of_sura_name','jomal_of_sura_name',\
        'jomal_of_sura_name_cumulative','unique_letters_of_sura_name','unique_letters_count_of_sura_name',\
            'unique_letters_invariant_of_sura_name','unique_letters_jomal_of_sura_name',\
                'unique_letters_jomal_of_sura_name_cumulative']]

# spoken text (Note: text as what it sounds) features

In [ ]:
quran['words_of_spoken_aya'] = quran['spoken_aya_text'].str.split()
quran['words_count_of_spoken_aya'] = quran['words_of_spoken_aya'].apply(lambda words: len(words))
quran['letters_of_spoken_aya'] = quran['spoken_aya_text'].apply(lambda aya: get_letters(aya))
quran['letters_count_of_spoken_aya'] = quran['letters_of_spoken_aya'].apply(lambda letters: len(letters))
quran['jomal_of_spoken_aya'] = quran['letters_of_spoken_aya'].apply(lambda letters: calculate_letters_jomal(letters))
quran['jomal_of_spoken_aya_cumulative'] = quran['jomal_of_spoken_aya'].cumsum()
quran['invariant_letters_of_spoken_aya'] = quran['spoken_aya_text'].apply(lambda letters: get_invariant_letters(letters))
quran['invariant_letters_count_of_spoken_aya'] = quran['invariant_letters_of_spoken_aya'].apply(lambda letters: len(letters))
quran['unique_letters_invariant_of_spoken_aya'] = quran['invariant_letters_of_spoken_aya'].apply(lambda letters: set(letters))
quran['unique_letters_invariant_count_of_spoken_aya'] = quran['unique_letters_invariant_of_spoken_aya'].apply(lambda letters: len(letters))
quran['unique_letters_of_spoken_aya'] = quran['letters_of_spoken_aya'].apply(lambda letters: set(letters))
quran['unique_letters_count_of_spoken_aya'] = quran['unique_letters_of_spoken_aya'].apply(lambda letters: len(letters))
quran['unique_letters_jomal_of_spoken_aya'] = quran['unique_letters_of_spoken_aya'].apply(lambda letters: calculate_letters_jomal(letters))
quran['unique_letters_cumulative_jomal_of_spoken_aya'] = quran['unique_letters_jomal_of_spoken_aya'].cumsum()
quran.head(10)[['words_of_spoken_aya','words_count_of_spoken_aya','letters_of_spoken_aya','letters_count_of_spoken_aya',\
    'jomal_of_spoken_aya','jomal_of_spoken_aya_cumulative','invariant_letters_of_spoken_aya',\
        'invariant_letters_count_of_spoken_aya','unique_letters_invariant_of_spoken_aya',\
            'unique_letters_invariant_count_of_spoken_aya','unique_letters_of_spoken_aya',\
                'unique_letters_count_of_spoken_aya','unique_letters_jomal_of_spoken_aya',\
                    'unique_letters_cumulative_jomal_of_spoken_aya']]

# written (uthmani) text features

In [ ]:
quran['words_of_written_aya'] = quran['written_aya_text'].apply(lambda x : get_written_aya_words(x))
quran['words_count_of_written_aya'] = quran['words_of_written_aya'].apply(lambda words: len(words))
quran['letters_of_written_aya'] = quran['written_aya_text'].apply(lambda aya: get_letters(aya))
quran['letters_count_of_written_aya'] = quran['letters_of_written_aya'].apply(lambda letters: len(letters))
quran['jomal_of_written_aya'] = quran['letters_of_written_aya'].apply(lambda letters: calculate_letters_jomal(letters))
quran['jomal_of_written_aya_cumulative'] = quran['jomal_of_written_aya'].cumsum()
quran['invariant_letters_of_written_aya'] = quran['written_aya_text'].apply(lambda letters: get_invariant_letters(letters))
quran['invariant_letters_count_of_written_aya'] = quran['invariant_letters_of_written_aya'].apply(lambda letters: len(letters))
quran['unique_letters_invariant_of_written_aya'] = quran['invariant_letters_of_written_aya'].apply(lambda letters: set(letters))
quran['unique_letters_invariant_count_of_written_aya'] = quran['unique_letters_invariant_of_written_aya'].apply(lambda letters: len(letters))
quran['unique_letters_of_written_aya'] = quran['letters_of_written_aya'].apply(lambda letters: set(letters))
quran['unique_letters_count_of_written_aya'] = quran['unique_letters_of_written_aya'].apply(lambda letters: len(letters))
quran['unique_letters_jomal_of_written_aya'] = quran['unique_letters_of_written_aya'].apply(lambda letters: calculate_letters_jomal(letters))
quran['unique_letters_cumulative_jomal_of_written_aya'] = quran['unique_letters_jomal_of_written_aya'].cumsum()
quran.head(10)[['words_of_written_aya','words_count_of_written_aya','letters_of_written_aya','letters_count_of_written_aya'\
    ,'jomal_of_written_aya','jomal_of_written_aya_cumulative','invariant_letters_of_written_aya',\
        'invariant_letters_count_of_written_aya','unique_letters_invariant_of_written_aya',\
            'unique_letters_invariant_count_of_written_aya','unique_letters_of_written_aya',\
                'unique_letters_count_of_written_aya','unique_letters_jomal_of_written_aya',\
                    'unique_letters_cumulative_jomal_of_written_aya']]

### fix spaces in words (61 places with not necessary places in plain text)

In [ ]:
# get un matched words counts between written and spoken text
words_to_fix = quran[(quran['words_count_of_spoken_aya'] != quran['words_count_of_written_aya'])]['words_of_spoken_aya']
len(words_to_fix)

In [ ]:
#fix spaces issue
words_to_fix.iloc[0][0:2] = [''.join(words_to_fix.iloc[0][0:2])]
words_to_fix.iloc[1][0:2] = [''.join(words_to_fix.iloc[1][0:2])]
words_to_fix.iloc[2][14:16] = [''.join(words_to_fix.iloc[2][14:16])]
words_to_fix.iloc[3][9:11] = [''.join(words_to_fix.iloc[3][9:11])]
words_to_fix.iloc[4][0:2] = [''.join(words_to_fix.iloc[4][0:2])]
words_to_fix.iloc[5][0:2] = [''.join(words_to_fix.iloc[5][0:2])]
words_to_fix.iloc[6][0:2] = [''.join(words_to_fix.iloc[6][0:2])]
words_to_fix.iloc[7][0:2] = [''.join(words_to_fix.iloc[7][0:2])]
words_to_fix.iloc[8][16:18] = [''.join(words_to_fix.iloc[8][16:18])]
words_to_fix.iloc[9][0:2] = [''.join(words_to_fix.iloc[9][0:2])]
words_to_fix.iloc[10][0:2] = [''.join(words_to_fix.iloc[10][0:2])]
words_to_fix.iloc[11][0:2] = [''.join(words_to_fix.iloc[11][0:2])]
words_to_fix.iloc[12][-4:-2] = [''.join(words_to_fix.iloc[12][-4:-2])]
words_to_fix.iloc[13][0:2] = [''.join(words_to_fix.iloc[13][0:2])]
words_to_fix.iloc[14][0:2] = [''.join(words_to_fix.iloc[14][0:2])]
words_to_fix.iloc[15][0:2] = [''.join(words_to_fix.iloc[15][0:2])]
words_to_fix.iloc[16][0:2] = [''.join(words_to_fix.iloc[16][0:2])]
words_to_fix.iloc[17][0:2] = [''.join(words_to_fix.iloc[17][0:2])]
words_to_fix.iloc[18][0:2] = [''.join(words_to_fix.iloc[18][0:2])]
words_to_fix.iloc[19][17:19] = [''.join(words_to_fix.iloc[19][17:19])]
words_to_fix.iloc[20][0:2] = [''.join(words_to_fix.iloc[20][0:2])]
words_to_fix.iloc[21][1:3] = [''.join(words_to_fix.iloc[21][1:3])]
words_to_fix.iloc[22][0:2] = [''.join(words_to_fix.iloc[22][0:2])]
words_to_fix.iloc[23][0:2] = [''.join(words_to_fix.iloc[23][0:2])]
words_to_fix.iloc[24][0:2] = [''.join(words_to_fix.iloc[24][0:2])]
words_to_fix.iloc[25][6:8] = [''.join(words_to_fix.iloc[25][6:8])]
words_to_fix.iloc[26][0:2] = [''.join(words_to_fix.iloc[26][0:2])]
words_to_fix.iloc[27][0:2] = [''.join(words_to_fix.iloc[27][0:2])]
words_to_fix.iloc[28][1:3] = [''.join(words_to_fix.iloc[28][1:3])]
words_to_fix.iloc[29][1:3] = [''.join(words_to_fix.iloc[29][1:3])]
words_to_fix.iloc[30][3:5] = [''.join(words_to_fix.iloc[30][3:5])]
words_to_fix.iloc[31][12:14] = [''.join(words_to_fix.iloc[31][12:14])]
words_to_fix.iloc[32][8:10] = [''.join(words_to_fix.iloc[32][8:10])]
words_to_fix.iloc[33][6:8] = [''.join(words_to_fix.iloc[33][6:8])]
words_to_fix.iloc[34][24:26] = [''.join(words_to_fix.iloc[34][24:26])]
words_to_fix.iloc[35][0:2] = [''.join(words_to_fix.iloc[35][0:2])]
words_to_fix.iloc[36][0:2] = [''.join(words_to_fix.iloc[36][0:2])]
words_to_fix.iloc[37][0:2] = [''.join(words_to_fix.iloc[37][0:2])]
words_to_fix.iloc[38][0:2] = [''.join(words_to_fix.iloc[38][0:2])]
words_to_fix.iloc[39][0:2] = [''.join(words_to_fix.iloc[39][0:2])]
words_to_fix.iloc[40][0:2] = [''.join(words_to_fix.iloc[40][0:2])]
words_to_fix.iloc[41][0:2] = [''.join(words_to_fix.iloc[41][0:2])]
words_to_fix.iloc[42][0:2] = [''.join(words_to_fix.iloc[42][0:2])]
words_to_fix.iloc[43][11:13] = [''.join(words_to_fix.iloc[43][11:13])]
words_to_fix.iloc[44][0:2] = [''.join(words_to_fix.iloc[44][0:2])]
words_to_fix.iloc[45][0:2] = [''.join(words_to_fix.iloc[45][0:2])]
words_to_fix.iloc[46][0:2] = [''.join(words_to_fix.iloc[46][0:2])]
words_to_fix.iloc[47][0:2] = [''.join(words_to_fix.iloc[47][0:2])]
words_to_fix.iloc[48][0:2] = [''.join(words_to_fix.iloc[48][0:2])]
words_to_fix.iloc[49][0:2] = [''.join(words_to_fix.iloc[49][0:2])]
words_to_fix.iloc[50][7:9] = [''.join(words_to_fix.iloc[50][7:9])]
words_to_fix.iloc[51][0:2] = [''.join(words_to_fix.iloc[51][0:2])]
words_to_fix.iloc[52][0:2] = [''.join(words_to_fix.iloc[52][0:2])]
words_to_fix.iloc[53][1:3] = [''.join(words_to_fix.iloc[53][1:3])]
words_to_fix.iloc[54][12:14] = [''.join(words_to_fix.iloc[54][12:14])]
words_to_fix.iloc[55][11:13] = [''.join(words_to_fix.iloc[55][11:13])]
words_to_fix.iloc[56][0:2] = [''.join(words_to_fix.iloc[56][0:2])]
words_to_fix.iloc[57][1:3] = [''.join(words_to_fix.iloc[57][1:3])]
words_to_fix.iloc[58][0:2] = [''.join(words_to_fix.iloc[58][0:2])]
words_to_fix.iloc[59][0:2] = [''.join(words_to_fix.iloc[59][0:2])]
words_to_fix.iloc[60][0:2] = [''.join(words_to_fix.iloc[60][0:2])]

In [ ]:
#recalculate words counts
quran['words_count_of_spoken_aya'] = quran['words_of_spoken_aya'].apply(lambda x: len(x))

In [ ]:
#check if there is any unmatched ayas left
len(quran[(quran['words_count_of_spoken_aya'] != quran['words_count_of_written_aya'])]['words_of_spoken_aya'])

### rearrange columns

In [ ]:
quran = quran [[\
'sura_name',\
'sura_mushaf_order',\
'sura_revel_order',\
'sura_revel_place',\
'sura_mushaf_order_cumulative',\
'sura_revel_order_cumulative',\
'letters_of_sura_name',\
'letters_count_of_sura_name',\
'invariant_letters_of_sura_name',\
'invariant_letters_count_of_sura_name',\
'jomal_of_sura_name',\
'jomal_of_sura_name_cumulative',\
'unique_letters_of_sura_name',\
'unique_letters_count_of_sura_name',\
'unique_letters_invariant_of_sura_name',\
'unique_letters_invariant_count_of_sura_name',\
'unique_letters_jomal_of_sura_name',\
'unique_letters_jomal_of_sura_name_cumulative',\
'aya_index',\
'aya_no',\
'written_aya_text',\
'words_of_written_aya',\
'words_count_of_written_aya',\
'letters_of_written_aya',\
'letters_count_of_written_aya',\
'jomal_of_written_aya',\
'jomal_of_written_aya_cumulative',\
'invariant_letters_of_written_aya',\
'invariant_letters_count_of_written_aya',\
'unique_letters_invariant_of_written_aya',\
'unique_letters_invariant_count_of_written_aya',\
'unique_letters_of_written_aya',\
'unique_letters_count_of_written_aya',\
'unique_letters_jomal_of_written_aya',\
'unique_letters_cumulative_jomal_of_written_aya',\
'spoken_aya_text',\
'words_of_spoken_aya',\
'words_count_of_spoken_aya',\
'letters_of_spoken_aya',\
'letters_count_of_spoken_aya',\
'jomal_of_spoken_aya',\
'jomal_of_spoken_aya_cumulative',\
'invariant_letters_of_spoken_aya',\
'invariant_letters_count_of_spoken_aya',\
'unique_letters_invariant_of_spoken_aya',\
'unique_letters_invariant_count_of_spoken_aya',\
'unique_letters_of_spoken_aya',\
'unique_letters_count_of_spoken_aya',\
'unique_letters_jomal_of_spoken_aya',\
'unique_letters_cumulative_jomal_of_spoken_aya'\
    ]]

In [ ]:
quran.info()

# save our dataset to new files

In [ ]:
quran.to_csv('generated_dataset/quran_Hafs_numeric_features.csv',index = False)
quran.to_excel('generated_dataset/quran_Hafs_numeric_features.xlsx',index = False)